In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from datetime import datetime, timedelta
from flask import Flask, request
from textblob import TextBlob

In [2]:
# download necessary nltk resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

app = Flask(__name__)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ajayraj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ajayraj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ajayraj\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# define function to extract relevant information from the input text
def extract_information(input_text):
    # define stop words to remove from text
    stop_words = set(stopwords.words('english'))

    # tokenize the input text
    words = word_tokenize(input_text)

    # remove stop words and punctuation
    filtered_words = [word for word in words if word.lower() not in stop_words and word.isalpha()]

    # part of speech tagging to identify nouns, verbs, adjectives, etc.
    pos_tags = pos_tag(filtered_words)

    # initialize variables to hold extracted information
    participants = []
    date = None
    time = None
    duration = None
    agenda = None

    # loop through pos tags to extract information
    for i in range(len(pos_tags)):
        word, pos = pos_tags[i]

        # extract participants
        if pos == 'NNP':
            participants.append(word)

        # extract date
        elif pos == 'CD':
            current_word, next_word = word, pos_tags[i+1][0]
            if current_word.lower() == 'next':
                if next_word.lower() == 'week':
                    date = (datetime.now() + timedelta(days=7)).strftime('%Y-%m-%d')
                elif next_word.lower() == 'wednesday':
                    date = (datetime.now() + timedelta(days=(2-datetime.now().weekday()))).strftime('%Y-%m-%d')
                else:
                    continue
            else:
                continue

        # extract time
        elif pos == 'JJ' and word.lower() == 'hour':
            if pos_tags[i-1][1] == 'CD':
                duration = int(pos_tags[i-1][0])
            else:
                continue
        elif pos == 'CD' and pos_tags[i+1][0].lower() == 'hour':
            duration = int(word)

        # extract agenda
        elif pos == 'TO' and pos_tags[i+1][1] == 'VB':
            agenda = ' '.join([word for word, pos in pos_tags[i+2:]])

    # return extracted information as a dictionary
    return {'participants': participants, 'date': date, 'time': time, 'duration': duration, 'agenda': agenda}

@app.route('/schedule', methods=['POST'])
def schedule():
    text = request.form['text']
    extracted_info = extract_information(text)
    # use extracted_info to schedule the meeting
    return "Meeting scheduled successfully!"

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
